# NASA Mars News

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import os
import pandas as pd
import time
import requests
import pymongo

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [3]:
db = client.mars_news
collection = db.articles

In [5]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
response = requests.get(url)
soup = bs(response.text, 'lxml')

In [28]:
#<div class="content_title" target="_self">/div>
#<div class="article_teaser_body"></div>
#<div class="list_text"> - parent???

news_title = soup.find("div", class_="content_title").text.strip()
news_paragraph = soup.find("div", class_="article_teaser_body")
print(f"News Title: {news_title}")
print(f"News Paragraph: {news_paragraph}")

News Title: Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
News Paragraph: None


# JPL Mars Space Images - Featured Image

In [33]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [34]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)


In [35]:
browser.click_link_by_id('full_image')

In [36]:
html = browser.html
image_soup = bs(html, "lxml")
img_url = image_soup.find('img', class_ = 'fancybox-image')['src']

In [37]:
featured_img_url = "https://www.jpl.nasa.gov" + img_url
featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16715_ip.jpg'